# **<h1><center><span style="color:#3775a8"><u>SORBONNE DATA ANALYTIC : Features Engineering</u></span></center></h1>**
# **<span style="color:#3775a8">Découverte de Pandas</span>**

**Bienvenue dans ce module !**

Pandas est l'un des packages les plus utilisées pour la manipulation de données en Python bien qu'il en existe d'autres (ex: datatable, vaex, ...).

Comme vous l'avez vu dans la vidéo, le features engineering permet de créer de l'information cachée dans vos données et sera souvent l'aspect différenciant de la puissance prédictive. Il n'existe pas de liste exhaustive d'informatoin à créer, le cours a permis d'appréhender des premières bases mais le spectre du features engineering est sans limites.

Vous allez voir dans cette formation comment utiliser pandas pour **créer de la valeur ajoutée à vos données** de manière à accroitre le pouvoir prédictif intitial.
Vous allez reprendre le travail précédent auquel vous allez rajouter de l'information issue de features engineering.

Avant toutes choses, voici les packages et les versions que j'utilise: 
- numpy==1.19.1
- pandas==1.1.0

**Attention**, si vous n'avez pas strictement les même, il est possible que vos output ne soient pas exactement les mêmes !

## **<span style="color:#3775a8">0. Importer les packages + bases de données</span>**

In [ ]:
import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')

path_data = '~/Mettez/Votre/Chemin/Data/'

In [ ]:
train = pd.read_csv(path_data + 'train.csv')
validation = pd.read_csv(path_data + 'validation.csv')
test = pd.read_csv(path_data + 'test.csv')

train['Date'] = pd.to_datetime(train['Date'], format="%Y-%m-%dT%H:%M:%S")
validation['Date'] = pd.to_datetime(validation['Date'], format="%Y-%m-%dT%H:%M:%S")
test['Date'] = pd.to_datetime(test['Date'], format="%Y-%m-%dT%H:%M:%S")

print(train.dtypes)
train.sample(n=10, random_state=893717398)

## **<span style="color:#3775a8">1. Calculs sur les dates</span>**
Vous allez créer 1 variable à partir de la date:
- la variable **Mois** représentant le mois de la date

In [ ]:
train['Mois'] = train['Date'].dt.month

## **<span style="color:#3775a8">2. Création de features à partir d'autres features</span>**
Vous allez créer 2 variables de ratio:
- la variable **Temp_Hum** représentant la température divisée par l'humidité
- la variable **Semestre** ayant la valeur 0 pour le premier semestre et 1 pour le second semestre construite à partir de **Mars**

In [ ]:
train['Temp_Hum'] = train['Temperature_dC'] / train['Humidite']
train['Semestre'] = np.where(train['Mois']<=6, 0, 1)

## **<span style="color:#3775a8">3. Discrétiser des variables continues</span>**
Si chaque ligne représent

In [ ]:
def discretise_temperature(x):
    if x < -5:
        return 'Tres_Froid'
    elif x < 0:
        return 'Froid_Negatif'
    elif x < 5:
        return 'Froid_Positif'
    elif x < 15:
        return 'Froid'
    elif x < 20:
        return 'Doux'
    elif x < 25:
        return 'Ideal'
    elif x < 30:
        return 'Chaud'
    elif x < 35:
        return 'Tres_Chaud'
    else:
        return 'Canicule'
    
def discretise_humidite(x):
    if x < 60:
        return 'Sec'
    if x < 96:
        return 'Humide'
    else:
        return 'Pluie'
    
train['Temperature_Discret'] = train['Temperature_dC'].apply(lambda x: discretise_temperature(x))
train['Humidite_Discret'] = train['Humidite'].apply(lambda x: discretise_humidite(x))

## **<span style="color:#3775a8">4. Agrégation de données</span>**
Ici vous allez agréger de l'information provenant des deux variables catégorielles que vous avez créé précedemment. Cette information représentera pour chaque paire possible la température médiane que vous stockerez dans une base de donnée appelée base_temperature_agregee.

In [ ]:
base_temperature_agregee = train.groupby(by=['Temperature_Discret', 'Humidite_Discret'])['Temperature_dC'].median().reset_index()
base_temperature_agregee.columns = ['Temperature_Discret', 'Humidite_Discret', 'Temperature_Agregee']

## **<span style="color:#3775a8">5. Ajouter des variables venant d'autres bases</span>**
A présent, vous allez fusionner votre base de train avec la base crée précédement et pour cela, la clé de jointure sera **Temperature_Discret** et **Humidite_Discret**.
Notez, que vous auriez pu ne pas passer par l'étape de merge pour cette tâche mais ici, l'objectif est de vous introduire et faire pratiquer cette notion de jointure.

In [ ]:
train= train.merge(base_temperature_agregee, on=['Temperature_Discret', 'Humidite_Discret'], how='left')

## **<span style="color:#3775a8">6. Création de features à partir d'autres features</span>**
Vous voyez qu'il est toujours possible de continuer de créer de nouvelles features et comme vous le savez à présent, le features engineering peut être très long car il n'a pas de limites!
Ici, vous allez créer le ratio **Variation_Temperature** entre la **Temperature_dC** et la **Temperature_Agregee** afin de voir où la température de l'observation se situe vis à vis d'une température médiane agrégée selon des niveaux de température et d'humidité généraux.

In [ ]:
train['Variation_Temperature'] = train['Temperature_dC'] / train['Temperature_Agregee']
train.sample(n=10, random_state=893717398)

## **<span style="color:#3775a8">7. Création de features automatiques</span>**
Je ne vais pas aborder ce point dans le notebook, mais pour rappel il existe des packages comme featuretools qui permettent de générer automatiquement une multitude de variables.


## **<span style="color:#3775a8">8. Les nouvelles features doivent être nettoyées</span>**
Au cours de ce notebook, vous avez créé 7 nouvelles variables alors qu'il n'y en avait que 3 qui étaient directement utilisables. Sachez que vous pourriez toujours en créer davantage comme par exemple en créant une variable liée à l'heure puis créer une variable catégorielle selon qu'il fasse jour ou nuit, etc. A présent, il est nécessaire de nettoyer ces nouvelles variables en suivant la méthodologie du cours précédent.

Vous pourrez vous exercés au nettoyage de ces nouvelles données en allant plus loin que précédement compte tenu que vous avez à présent des variables catégorielles, où vous pourrez faire des regroupements, tester les corrélations, encoder, etc.
Pour rappel, seulement les variables numériques pourront être utilisées pour faire la modélisation !

# **<span style="color:#37a871">Félicitations !</span>**